In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\Lenovo\\Desktop\\Portfolio-Optimization-Using-Markowitz-Model'

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [64]:
number_of_assets=5

In [65]:
number_of_observations=4000

In [66]:
return_vec= np.random.randn(number_of_assets, number_of_observations)

In [67]:
return_vec

array([[-0.25926575,  0.03468524, -1.27949175, ...,  0.45194392,
         0.75410883,  0.1502216 ],
       [ 1.20995753, -1.50678004, -0.60224442, ..., -0.83153544,
        -0.07686055,  0.04270244],
       [ 1.37731859, -0.36596929,  1.18287955, ..., -0.195933  ,
         0.61668928, -0.41574978],
       [ 0.30312251,  1.27530486,  1.48088105, ...,  0.4611618 ,
        -0.22605006, -0.98518136],
       [ 0.85175433, -0.54759924,  1.64188757, ..., -1.46390888,
         1.19958159, -0.47802237]])

In [49]:
def rand_weights(n):
    k = np.random.rand(n)
    return (k)/(sum(k))

In [50]:
def random_portfolio(returns):
    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    return mu, sigma

In [68]:
number_of_portfolio=1000

In [69]:
while True:
    means, stds = np.column_stack([
        random_portfolio(return_vec) 
        for _ in range(number_of_portfolio)
        ])
    if stds.all() <= 2:
        break

In [70]:
means

array([[ 7.46228623e-03],
       [-2.14821612e-03],
       [-1.03465031e-02],
       [ 1.18090945e-02],
       [ 1.14708414e-02],
       [-7.66713581e-03],
       [-1.15855745e-02],
       [-8.10549527e-03],
       [-2.45811004e-04],
       [ 4.99836713e-03],
       [-8.68573330e-03],
       [ 1.29080566e-03],
       [-4.62141814e-03],
       [ 2.74851138e-03],
       [-1.43200672e-03],
       [ 1.06928611e-02],
       [ 4.28046307e-04],
       [-1.16836137e-02],
       [ 9.36230256e-04],
       [-5.06579060e-03],
       [ 4.64979089e-03],
       [ 1.48841768e-02],
       [ 9.59437805e-03],
       [-1.92612983e-03],
       [ 7.23510913e-03],
       [-1.52995103e-03],
       [ 7.90662333e-03],
       [ 4.31508137e-03],
       [-1.06784791e-02],
       [ 1.31931772e-02],
       [ 1.27230254e-02],
       [ 1.16795176e-03],
       [ 6.81092633e-03],
       [ 8.48518831e-03],
       [ 1.66879174e-03],
       [-1.17874461e-02],
       [ 7.52837085e-03],
       [ 7.60521023e-03],
       [ 1.1

In [71]:
stds

array([[0.51410179],
       [0.51515205],
       [0.53327887],
       [0.51460195],
       [0.52379519],
       [0.55410455],
       [0.53466969],
       [0.5425735 ],
       [0.49796363],
       [0.46578517],
       [0.52781426],
       [0.46682878],
       [0.50819198],
       [0.48867953],
       [0.49273489],
       [0.4873356 ],
       [0.45222592],
       [0.52873587],
       [0.48467622],
       [0.52658707],
       [0.54810615],
       [0.55988611],
       [0.50116306],
       [0.49688143],
       [0.54948633],
       [0.49411226],
       [0.52811037],
       [0.47376453],
       [0.54593064],
       [0.54187864],
       [0.57883992],
       [0.45827223],
       [0.49166447],
       [0.54216765],
       [0.49921195],
       [0.57602356],
       [0.48043576],
       [0.46981978],
       [0.51275304],
       [0.49598141],
       [0.48402638],
       [0.52469912],
       [0.49916259],
       [0.49193912],
       [0.47900313],
       [0.52090677],
       [0.49291995],
       [0.559

In [72]:
len(stds)

1000

In [73]:
len(means)

1000

In [74]:
import plotly.graph_objects as go

In [75]:
fig = go.Figure()
fig.update_layout(xaxis_title='Standard Deviation', 
                  yaxis_title='Mean')
fig.add_trace(go.Scatter(x=stds.flatten(), y=means.flatten(),mode='markers'))
fig.show()

In [19]:
import cvxopt as opt

In [20]:
from cvxopt import blas, solvers

In [21]:
def optimize(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] for mu in mus]
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    
    return np.asarray(wt), returns, risks

In [76]:
weights, returns, risks = optimize(return_vec)

     pcost       dcost       gap    pres   dres
 0:  6.3958e-03 -1.0189e+00  1e+00  1e-16  3e+00
 1:  5.6406e-03 -2.7970e-02  3e-02  2e-16  9e-02
 2: -1.1791e-03 -5.7707e-03  5e-03  2e-16  2e-17
 3: -2.5655e-03 -2.8868e-03  3e-04  2e-16  5e-18
 4: -2.7066e-03 -2.7141e-03  7e-06  2e-16  6e-18
 5: -2.7084e-03 -2.7085e-03  8e-08  2e-16  1e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  7.6590e-03 -1.0174e+00  1e+00  0e+00  3e+00
 1:  6.9236e-03 -2.6309e-02  3e-02  9e-17  9e-02
 2:  7.0994e-04 -3.4532e-03  4e-03  2e-16  2e-17
 3: -5.7186e-04 -8.6207e-04  3e-04  1e-16  8e-18
 4: -7.2058e-04 -7.2475e-04  4e-06  1e-16  1e-17
 5: -7.2275e-04 -7.2279e-04  4e-08  1e-16  8e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  9.0750e-03 -1.0156e+00  1e+00  0e+00  3e+00
 1:  8.3608e-03 -2.4456e-02  3e-02  7e-17  9e-02
 2:  2.7342e-03 -1.0281e-03  4e-03  2e-16  4e-05
 3:  1.5637e-03  1.3020e-03  3e-04  1e-16  9e-07
 4:  1.4037e-03  1.3995e

 4:  2.2426e+01  2.2426e+01  1e-08  1e-16  3e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.5163e+01  2.4154e+01  1e+00  2e-16  3e+00
 1:  2.5163e+01  2.5152e+01  1e-02  9e-17  3e-02
 2:  2.5163e+01  2.5162e+01  1e-04  1e-16  3e-04
 3:  2.5163e+01  2.5163e+01  1e-06  2e-16  3e-06
 4:  2.5163e+01  2.5163e+01  1e-08  1e-16  3e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.8233e+01  2.7225e+01  1e+00  0e+00  3e+00
 1:  2.8233e+01  2.8223e+01  1e-02  6e-17  3e-02
 2:  2.8233e+01  2.8233e+01  1e-04  9e-17  3e-04
 3:  2.8233e+01  2.8233e+01  1e-06  3e-17  3e-06
 4:  2.8233e+01  2.8233e+01  1e-08  9e-17  3e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  3.1678e+01  3.0670e+01  1e+00  2e-16  3e+00
 1:  3.1678e+01  3.1668e+01  1e-02  6e-17  3e-02
 2:  3.1678e+01  3.1678e+01  1e-04  1e-16  3e-04
 3:  3.1678e+01  3.1678e+01  1e-06  4e-17  3e-06
 4:  3.1678e+01  3.1678e+01  1e-08  1e-16  3e-08


In [59]:
len(returns)

100

In [77]:
fig = go.Figure()
fig.update_layout(xaxis_title='Standard Deviation', 
                  yaxis_title='Mean',
                  showlegend=False)
fig.add_trace(go.Scatter(x=stds.flatten(), y=means.flatten(),mode='markers'))
fig.add_trace(go.Scatter(x=risks,y=returns,mode='markers+lines',name='Efficient Frontier'))
fig.show()

In [78]:
weights

array([[7.41881179e-01],
       [2.27846533e-06],
       [2.58116492e-01],
       [4.60261471e-08],
       [4.30464084e-09]])

In [79]:
sum_weights=0
for each in weights:
    sum_weights+=each[0]

In [80]:
sum_weights

1.0